In [14]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("redwankarimsony/heart-disease-data")

print("Path to dataset files:", path)

import pandas as pd
df=pd.read_csv('../datasets/heart_disease_uci.csv')
df


Path to dataset files: /Users/nikodemkaczmarek/.cache/kagglehub/datasets/redwankarimsony/heart-disease-data/versions/6


,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,916,54,Female,VA Long Beach,asymptomatic,127.0,333.0,True,st-t abnormality,154.0,False,0.0,NaN,NaN,NaN,1
916,917,62,Male,VA Long Beach,typical angina,NaN,139.0,False,st-t abnormality,NaN,NaN,NaN,NaN,NaN,NaN,0
917,918,55,Male,VA Long Beach,asymptomatic,122.0,223.0,True,st-t abnormality,100.0,False,0.0,NaN,NaN,fixed defect,2
918,919,58,Male,VA Long Beach,asymptomatic,NaN,385.0,True,lv hypertrophy,NaN,NaN,NaN,NaN,NaN,NaN,0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# -------------------------------
# 1. Load dataset
file_path = "../datasets/heart_disease_uci.csv"
df = pd.read_csv(file_path)
print("Initial dataset shape:", df.shape)

# -------------------------------
# 2. Drop irrelevant columns
df = df.drop(['id','dataset'], axis=1)
df.drop(columns=['thal','ca'], inplace = True)

# -------------------------------
# 3. Handle missing values
# Fill numeric columns with median
df.loc[df['trestbps'] == 0, 'trestbps'] = np.nan
df.loc[df['chol'] == 0, 'chol'] = np.nan

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
print('numeric: ', numeric_cols)
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())


# Fill categorical columns with mode
categorical_cols = df.select_dtypes(include=['object']).columns
print('categorical: ', categorical_cols)
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

# -------------------------------
# 4. Encode categorical variables (one-hot encoding)
df = pd.get_dummies(df, drop_first=True)
bool_cols = df.select_dtypes(bool).columns
df[bool_cols] = df[bool_cols].astype(int)


# -------------------------------
# 5. Split features and target
X = df.drop('num', axis=1)
y = df['num']

# -------------------------------
# 6. Feature scaling
scaler = StandardScaler()
# Scale only numeric columns (excluding 'num')
numeric_cols = numeric_cols.drop('num')
X_scaled_part = scaler.fit_transform(X[numeric_cols])

# Replace the numeric columns in X with their scaled versions
X_scaled = X.copy()
X_scaled[numeric_cols] = X_scaled_part

# -------------------------------
# 7. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# -------------------------------
# 8. Handle class imbalance on training set using SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train) # type: ignore

# -------------------------------
# 9. Final dataset info
print("Original training set shape:", X_train.shape, y_train.shape)
print("Resampled training set shape:", X_train_res.shape, y_train_res.shape)
print("Test set shape:", X_test.shape, y_test.shape)

# Optional: check target distribution after resampling
print(pd.Series(y_train_res).value_counts()) # type: ignore


Initial dataset shape: (920, 16)
numeric:  Index(['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'num'], dtype='object')
categorical:  Index(['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope'], dtype='object')
Original training set shape: (736, 15) (736,)
Resampled training set shape: (1645, 15) (1645,)
Test set shape: (184, 15) (184,)
num
3    329
0    329
2    329
1    329
4    329
Name: count, dtype: int64


/var/folders/hy/098gq1ks5j7f1bns6z1_ghbw0000gn/T/ipykernel_65803/2380034988.py:33: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[col].mode()[0])


In [17]:
df

,age,trestbps,chol,fbs,thalch,exang,oldpeak,num,sex_Male,cp_atypical angina,cp_non-anginal,cp_typical angina,restecg_normal,restecg_st-t abnormality,slope_flat,slope_upsloping
0,63,145.0,233.0,True,150.0,False,2.3,0,True,False,False,True,False,False,False,False
1,67,160.0,286.0,False,108.0,True,1.5,2,True,False,False,False,False,False,True,False
2,67,120.0,229.0,False,129.0,True,2.6,1,True,False,False,False,False,False,True,False
3,37,130.0,250.0,False,187.0,False,3.5,0,True,False,True,False,True,False,False,False
4,41,130.0,204.0,False,172.0,False,1.4,0,False,True,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,54,127.0,333.0,True,154.0,False,0.0,1,False,False,False,False,False,True,True,False
916,62,130.0,139.0,False,140.0,False,0.5,0,True,False,False,True,False,True,True,False
917,55,122.0,223.0,True,100.0,False,0.0,2,True,False,False,False,False,True,True,False
918,58,130.0,385.0,True,140.0,False,0.5,0,True,False,False,False,False,False,True,False


In [20]:
bool_cols

Index(['fbs', 'exang', 'sex_Male', 'cp_atypical angina', 'cp_non-anginal',
       'cp_typical angina', 'restecg_normal', 'restecg_st-t abnormality',
       'slope_flat', 'slope_upsloping'],
      dtype='object')

In [21]:
import os

out_dir = "../data"
os.makedirs(out_dir, exist_ok=True)

# 1) Save processed dataframe (after get_dummies)
df.to_csv(os.path.join(out_dir, "heart_df_processed.csv"), index=False)

# 2) Save full scaled features + target
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
y_reset = y.reset_index(drop=True)
Xy_scaled = pd.concat([X_scaled_df, y_reset], axis=1)
Xy_scaled.to_csv(os.path.join(out_dir, "heart_Xy_scaled.csv"), index=False)

# 3) Save train/test sets (scaled)
X_train_df = pd.DataFrame(X_train, columns=X.columns)
X_test_df = pd.DataFrame(X_test, columns=X.columns)
pd.Series(y_train, name='num').to_csv(os.path.join(out_dir, "y_train.csv"), index=False)
pd.Series(y_test, name='num').to_csv(os.path.join(out_dir, "y_test.csv"), index=False)
X_train_df.to_csv(os.path.join(out_dir, "X_train.csv"), index=False)
X_test_df.to_csv(os.path.join(out_dir, "X_test.csv"), index=False)

# 4) Save resampled training set (after SMOTE)
X_train_res_df = pd.DataFrame(X_train_res, columns=X.columns)
y_train_res_df = pd.Series(y_train_res, name='num') # type: ignore
Xy_train_res = pd.concat([X_train_res_df, y_train_res_df.reset_index(drop=True)], axis=1)
Xy_train_res.to_csv(os.path.join(out_dir, "Xy_train_resampled.csv"), index=False)

print("Saved CSVs to", out_dir)


Saved CSVs to ../data
